## Transfer Learning using InceptionV3 and Keras

In [26]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [28]:
!rm -rf ./logs/ 

In [29]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/train'
valid_path = '/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/Validation'
trial_path='/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/test'


In [32]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [33]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [34]:
  # useful for getting number of output classes
folders = glob('/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/train/*')

In [35]:
folders

['/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/train/rough_train',
 '/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/train/slippery_train']

In [36]:
x = Flatten()(vgg16.output)

In [37]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [38]:

# view the structure of the model
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [39]:
# tell the model what cost and optimization method to use
model.compile(
 loss='binary_crossentropy',
  optimizer='Adam',
  metrics=['accuracy']
  
)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [40]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator;

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,  
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,  
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [41]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1281 images belonging to 2 classes.


In [42]:
training_set

In [43]:
test_set = test_datagen.flow_from_directory('/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/Validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 328 images belonging to 2 classes.


In [44]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/Othercomputers/My Laptop/artefical intellegence/mydataset/test",
    image_size = (224, 224),
    batch_size = 32 )

Found 31 files belonging to 2 classes.


In [45]:
class_names = ["rough", "slippery"]

In [46]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, 
        verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[tensorboard_callback, monitor]
)

<ipython-input-47-77b497bd5165>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
41/41 [==============================] - 880s 22s/step - loss: 0.4797 - accuracy: 0.7603 - val_loss: 0.6192 - val_accuracy: 0.7409
Epoch 2/50
41/41 [==============================] - 876s 21s/step - loss: 0.3258 - accuracy: 0.8517 - val_loss: 0.5980 - val_accuracy: 0.7500
Epoch 3/50
41/41 [==============================] - 878s 22s/step - loss: 0.3109 - accuracy: 0.8618 - val_loss: 0.5239 - val_accuracy: 0.7927
Epoch 4/50
41/41 [==============================] - 871s 21s/step - loss: 0.2912 - accuracy: 0.8735 - val_loss: 0.4812 - val_accuracy: 0.8140
Epoch 5/50
41/41 [==============================] - 872s 21s/step - loss: 0.2457 - accuracy: 0.9009 - val_loss: 0.5048 - val_accuracy: 0.8354
Epoch 6/50
32/41 [======================>.......] - ETA: 2:25 - loss: 0.2072 - accuracy: 0.9204

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.evaluate(test_ds)

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
  classifications = model(images)
  # print(classifications)
  
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + " | Real: " + class_names[labels[i]])